In [6]:
import json
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import pad_sequences
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np
features_d2 = []
labels_d2 = []
# Open file for reading
with open('domain2_train.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        features_d2.append(obj['text'])
        labels_d2.append(obj['label'])

In [7]:
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  

# Fit and transform the data to text
features_tfidf = tfidf.fit_transform(features_d2)
label_d2 = np.array(labels_d2)

In [99]:

import pickle

# Save the trained vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)


In [8]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_tfidf, labels_d2, test_size=0.2, random_state=42)


In [4]:
import lightgbm as lgb

In [56]:
from skopt import BayesSearchCV
import lightgbm as lgb

from sklearn.metrics import accuracy_score


# Define the simplified hyperparameter configuration space
param_space = {
    'num_leaves': (25, 50),
    'max_depth': (10, 30),
    'min_child_samples': (35, 60),
    'subsample': (0.7, 1.0),
    'colsample_bytree': (0.7, 1.0),
    'scale_pos_weight': (3, 15),
    'learning_rate': (0.01, 0.1, 'log-uniform'),
    'n_estimators': (100, 1000),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0),
}




opt = BayesSearchCV(
    estimator=lgb.LGBMClassifier(
        objective='binary',
        metric='binary_error',
        n_jobs=2,
        verbose= -1
    ),
    search_spaces=param_space,
    scoring='f1',
    cv=5,
    n_jobs=1,
    n_iter=30,
    verbose=100,
    refit=True,
)

opt.fit(features_tfidf, labels_d2)



Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START colsample_bytree=0.7852647356757848, learning_rate=0.013203027910561781, max_depth=27, min_child_samples=42, n_estimators=285, num_leaves=30, reg_alpha=0.875386340044103, reg_lambda=0.7290703968971941, scale_pos_weight=12, subsample=0.7511013826320925
[CV 1/5; 1/1] END colsample_bytree=0.7852647356757848, learning_rate=0.013203027910561781, max_depth=27, min_child_samples=42, n_estimators=285, num_leaves=30, reg_alpha=0.875386340044103, reg_lambda=0.7290703968971941, scale_pos_weight=12, subsample=0.7511013826320925;, score=0.353 total time=   9.4s
[CV 2/5; 1/1] START colsample_bytree=0.7852647356757848, learning_rate=0.013203027910561781, max_depth=27, min_child_samples=42, n_estimators=285, num_leaves=30, reg_alpha=0.875386340044103, reg_lambda=0.7290703968971941, scale_pos_weight=12, subsample=0.7511013826320925
[CV 2/5; 1/1] END colsample_bytree=0.7852647356757848, learning_rate=0.013203027910561781, max

BayesSearchCV(cv=5,
              estimator=LGBMClassifier(metric='binary_error', n_jobs=2,
                                       objective='binary', verbose=-1),
              n_iter=30, scoring='f1',
              search_spaces={'colsample_bytree': (0.7, 1.0),
                             'learning_rate': (0.01, 0.1, 'log-uniform'),
                             'max_depth': (10, 30),
                             'min_child_samples': (35, 60),
                             'n_estimators': (100, 1000),
                             'num_leaves': (25, 50), 'reg_alpha': (0.0, 1.0),
                             'reg_lambda': (0.0, 1.0),
                             'scale_pos_weight': (3, 15),
                             'subsample': (0.7, 1.0)},
              verbose=100)

In [63]:
opt.best_score_

0.3997561637815428

In [62]:
opt.best_params_

OrderedDict([('colsample_bytree', 0.7234310885166573),
             ('learning_rate', 0.016423885345104958),
             ('max_depth', 23),
             ('min_child_samples', 60),
             ('n_estimators', 470),
             ('num_leaves', 50),
             ('reg_alpha', 0.30421732656825395),
             ('reg_lambda', 0.0),
             ('scale_pos_weight', 8),
             ('subsample', 0.9344694754621987)])

In [98]:
best_params = opt.best_params_





best_params['scale_pos_weight'] = 7.5

# Manually train a model using the best params
clf = lgb.LGBMClassifier(**best_params)
clf.fit(features_tfidf, labels_d2)



# Save model
clf.booster_.save_model('domain2_model.txt')



In [83]:
y_test

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [82]:
y_pred_binary

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


In [65]:
y_pred

array([0.15120917, 0.35533517, 0.28387424, ..., 0.65429229, 0.64706411,
       0.5465303 ])

In [70]:

accuracy = accuracy_score(y_test, y_pred_binary)
print("precision:", accuracy)

precision: 0.7751677852348994


In [66]:
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

In [67]:
recall_score(y_test, y_pred_binary)

0.5504587155963303

In [33]:
y_pred

array([0, 1, 0, ..., 1, 0, 1])

In [61]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, f1_score
import lightgbm as lgb

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)




best_params = {
    'colsample_bytree': 0.72,
    'learning_rate': 0.01642,
    'max_depth': 23,
    'min_child_samples': 60,
    'num_leaves': 50,
    'n_estimators': 470,
    'scale_pos_weight': 8,
    'subsample': 0.93446,
    'reg_alpha': 0.30421732656825395

}
bst = lgb.train(best_params, train_data, valid_sets=[test_data])

# Make predictions
y_pred = bst.predict(X_test)
# Convert to binary output
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

# Evaluate the model
f1s = f1_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
auc = roc_auc_score(y_test, y_pred)  # y_pred is the probability, not the binary output

print(f"f1s: {f1s}")
print(f"Precision: {precision}")
print(f"AUC: {auc}")

c:\Users\chenj\anaconda3\envs\CV\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.095362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 162112
[LightGBM] [Info] Number of data points in the train set: 11920, number of used features: 2171
[LightGBM] [Info] Start training from score 0.143792
[1]	valid_0's l2: 0.12454
[2]	valid_0's l2: 0.124207
[3]	valid_0's l2: 0.123862
[4]	valid_0's l2: 0.123668
[5]	valid_0's l2: 0.12336
[6]	valid_0's l2: 0.123078
[7]	valid_0's l2: 0.122752
[8]	valid_0's l2: 0.122563
[9]	valid_0's l2: 0.122279
[10]	valid_0's l2: 0.122182
[11]	valid_0's l2: 0.121908
[12]	valid_0's l2: 0.121738
[13]	valid_0's l2: 0.12146
[14]	valid_0's l2: 0.121204
[15]	valid_0's l2: 0.120948
[16]	valid_0's l2: 0.120745
[17]	valid_0's l2: 0.120582
[18]	valid_0's l2: 0.120358
[19]	valid_0's l2: 0.120078
[20]	valid_0's l2: 0.11981
[21]	valid_0's l2: 0.119687
[22]	valid_0

In [ ]:
# Make predictions
y_pred = opt.predict(X_test)

# Evaluate the model
precision = precision_score(y_test, y_pred_binary)
print(f"Precision: {precision}")

Precision: 0.7142857142857143


In [60]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred_binary)

0.052752293577981654

In [ ]:
precision

1.0